In [ ]:
import time
import numpy as np
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import math
from PIL import Image
import random
import tensorflow_hub as hub
import keras

def saveQ_table(Q_table,name):
    np.save(str(name)+".npy", Q_table)

def openQ_table(name):
    return np.load(str(name)+".npy", allow_pickle=True)


from google.colab import drive
drive.mount("/content/drive/")


def saveQ_table(Q_table,name):
    np.save(str(name)+".npy", Q_table)

def openQ_table(name):
    return np.load(str(name)+".npy", allow_pickle=True)


def saveQ_table_Drive(Q_table,name):
    np.save("drive/MyDrive/Colab Notebooks/RouletteData/"+str(name)+".npy", Q_table)

def openQ_table_Drive(name):
    return np.load("drive/MyDrive/Colab Notebooks/RouletteData/"+str(name)+".npy", allow_pickle=True)




Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def tierNombre(nbr):
  if nbr > 0 and nbr < 13 :
    return 1
  elif nbr > 12 and nbr < 25 :
    return 2 
  elif nbr > 24 and nbr < 37 :
    return 3
  elif nbr ==0 :
    return random.randint(1,3)


def halfNombre(nbr):
  if nbr > 0 and nbr < 19 :
    return 1
  elif nbr > 18 and nbr < 37 :
    return 2 
  elif nbr ==0 :
    return random.randint(1,2)

def retour_proba_tiers ( proba):
  retour =[0,0,0]
  for i in range(1,len(proba)) :
    retour[tierNombre(i)-1] +=proba[i]

  return retour


def probaToNombre(proba):
    tab_Nbr=[ 0,0,0, 0,0,0 ]


    for i in range(len(proba)):

        if proba[i]>=30 and proba[i]<= 70 and i!=0 :
            tab_Nbr[tierNombre(i)-1]+= 1
            # proba[i]

        elif proba[i]>= 80 and proba[i]<= 100 and i!=0 :
            tab_Nbr[tierNombre(i)-1+3]+=1   


        # elif proba[i]>= 70 and proba[i]<= 100 and i!=0 :
        #     tab_Nbr[tierNombre(i)-1+3]+=1   




    return tab_Nbr


In [ ]:
def vecteurs_direction(raw_data,tier) :
    tableau_proba=[]
    tableau_label=[]
    tableau_vect=[]
    for i in range(0,len(raw_data)-60,2 ) :

        vect_direction=[0,0,0]

        for tiers in range(3) :
            condition_tiers=False
            cpt=0
            while condition_tiers ==False :
                if tierNombre(raw_data[i+1 +cpt])!= tiers+1 :
                    vect_direction[tiers] +=1
                elif tierNombre(raw_data[i+1 +cpt]) == tiers+1 :
                    condition_tiers=True

                cpt+=2


        tableau_proba.append( retour_proba_tiers( raw_data[i]) )
        tableau_label.append(tierNombre( raw_data[i+1] ) )
        tableau_vect.append( np.argmax( vect_direction ) )

    # for i in range(len(tableau_vect)) :
    #     for j in range(len(tableau_vect[i])) :
    #         if tableau_vect[i][j] <=-7 :
    #           tableau_vect[i][j]=0
    #         else :
    #           tableau_vect[i][j]=1


    # for i in range(len(tableau_vect)) :
    #     if tableau_vect[i] >= 10 :
    #       tableau_vect[i] =1
    #     else :
    #       tableau_vect[i] =0


    return tableau_proba, tableau_label , tableau_vect

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  labels = tf.cast( labels, tf.float32 )
  # norme= tf.norm(labels,ord="euclidean" )
  # labels /= norme
  images /= 100*12
  return images, labels


In [ ]:
raw_data_roulette=openQ_table_Drive("Last_Save")


In [ ]:
# raw_data=[]
# raw_label=[]
# for i in range( 0,len(raw_data_roulette),2) :
#     raw_data.append( retour_proba_tiers(raw_data_roulette[i]) )
#     raw_label.append( tierNombre(raw_data_roulette[i+1]) -1 )

# raw_proba=raw_data
# raw_vecteur=raw_label

raw_proba , raw_label , raw_vecteur = vecteurs_direction(raw_data_roulette,1)

In [ ]:
print(len(raw_proba), len(raw_label) )

61394 61394


In [ ]:
test_data_raw , test_label_raw = raw_proba[-1000:] , raw_vecteur[-1000:]
# validation_data,validation_label =raw_data[-2000:-1000] , raw_label[-2000:-1000]
training_data_raw , training_label_raw = raw_proba[0:-1000] , raw_vecteur[0:-1000]
print("longueur test_data : ",len(test_data_raw),"longueur test_label : ",len(test_label_raw))
# print("longueur validation_data : ",len(validation_data),"longueur validation_label : ",len(validation_label))
print("longueur training_data : ",len(training_data_raw),"longueur training_label : ",len(training_label_raw))

longueur test_data :  1000 longueur test_label :  1000
longueur training_data :  60394 longueur training_label :  60394


In [ ]:
test_data_raw[0]

[587.75, 535.03, 691.76]

In [ ]:
test_data ,test_label = np.reshape( raw_proba[-1000:] ,(len((raw_proba[-1000:])), 3,1,1 )) , raw_vecteur[-1000:]
# validation_data,validation_label = np.reshape( raw_data[-2000:-1000] ,(len(raw_data[-2000:-1000]), 37,1,1 )) , raw_label[-2000:-1000]
training_data,training_label = np.reshape( raw_proba[0:-1000] ,( len(raw_proba[0:-1000] ), 3,1,1 ) ) , raw_vecteur[0:-1000]
print("longueur test_data : ",len(test_data),"longueur test_label : ",len(test_label))
# print("longueur validation_data : ",len(validation_data),"longueur validation_label : ",len(validation_label))
print("longueur training_data : ",len(training_data),"longueur training_label : ",len(training_label))

longueur test_data :  1000 longueur test_label :  1000
longueur training_data :  60394 longueur training_label :  60394


In [ ]:
np.shape(test_data_raw)
# test_data[0][0][0][0]

(1000, 3)

In [ ]:
test_dataset= tf.data.Dataset.from_tensor_slices( (test_data,test_label ) ).map(normalize)
# validation_dataset= tf.data.Dataset.from_tensor_slices( ( validation_data, validation_label ) ).map(normalize)
training_dataset= tf.data.Dataset.from_tensor_slices( (training_data , training_label ) ).map(normalize)

In [ ]:
BACTH_SIZE=32
test_dataset = test_dataset.cache().batch(BACTH_SIZE)
# validation_dataset = validation_dataset.batch(BACTH_SIZE)
training_dataset = training_dataset.batch(BACTH_SIZE)

print(len(test_dataset))
# print(len(validation_dataset))
print(len(training_dataset))

32
1888


In [ ]:
for img , lab in test_dataset.take(1) :
    break

print( lab )

tf.Tensor(
[0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 1. 1. 2. 0. 1. 1. 1. 1. 1. 1. 1. 2. 0. 0.
 0. 0. 1. 1. 1. 2. 2. 1.], shape=(32,), dtype=float32)


In [ ]:
# for num , tier in training_dataset.take(1) :
#     # print(num)
#     print(num)



In [ ]:
len(training_dataset)

1888

In [ ]:
model_Test_1 = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 128 , ( 1 ,1), padding='same', activation= tf.nn.leaky_relu ,
                           input_shape=(3,1 ,1) ) ,
    tf.keras.layers.Conv2D( 256 , ( 1 ,1 ), padding='same', activation= tf.nn.leaky_relu ) ,
    tf.keras.layers.Flatten() ,
    tf.keras.layers.Dense( 512 , activation= tf.nn.leaky_relu  ) ,
    tf.keras.layers.Dense( 3 , activation=tf.nn.softmax )

] )

In [ ]:
# model_Test_1 = tf.keras.Sequential( [
#     tf.keras.layers.Flatten( input_shape=(3 ,1 ,1) ),
#     tf.keras.layers.Dense(128 , activation=tf.keras.activations.linear ),
#     tf.keras.layers.Dense( 3 , activation=tf.keras.activations.linear )

# ] )

In [ ]:
model_Test_1.compile( optimizer='adam',
              loss= tf.keras.losses.Huber() ,
              # loss= tf.keras.losses.SparseCategoricalCrossentropy() ,
              metrics=['accuracy'] )
              # metrics=['mae'] )


In [ ]:
# model_Test_1=tf.keras.models.load_model("model_1000epoch.h5")

In [ ]:
model_Test_1.fit( training_dataset, epochs = 1 , steps_per_epoch=math.ceil( len(training_data)/BACTH_SIZE ) , shuffle=True)

1888/1888 [==============================] - 14s 7ms/step - loss: 0.4777 - accuracy: 0.3264


In [ ]:
test_loss , test_accuracy = model_Test_1.evaluate( test_dataset , steps=math.ceil(len(test_data)/BACTH_SIZE))
print('Accuracy on test dataset:', test_accuracy )

32/32 [==============================] - 0s 4ms/step - loss: 0.4242 - accuracy: 0.3360
Accuracy on test dataset: 0.335999995470047


In [ ]:
test_loss , test_accuracy = model_Test_1.evaluate( training_dataset , steps=math.ceil(len(test_data)/BACTH_SIZE))
print('Accuracy on test dataset:', test_accuracy )

32/32 [==============================] - 0s 6ms/step - loss: 0.5071 - accuracy: 0.3105
Accuracy on test dataset: 0.310546875


In [ ]:
len(test_label_raw)

1000

In [ ]:
# temp_loaded= tf.keras.models.load_model("model_3410.h5")
# temp_loaded.summary()

In [ ]:
def diviser_par_cent(test_data_raw) :
  temp_test_data=[]
  for i in range( len(test_data_raw) ) :
      temp=[]
      for j in range(len(test_data_raw[i])) :
          temp.append( test_data_raw[i][j]/100 )
      temp_test_data.append(temp)
  return temp_test_data

temp_test_data=diviser_par_cent(test_data_raw)

# for i in range(300,301) :
#     print( np.argmax( model_Test_1.predict( np.reshape( temp_test_data[i],(1,37,1,1)) ) ) +1 , test_label_raw[i] +1)

In [ ]:
# temp_test_data[0]

In [ ]:
b= 400
for i in range(b,b+100) :
    print( np.argmax( model_Test_1.predict( np.reshape( temp_test_data[i],(1,3,1,1)) ) ) , test_label_raw[i], model_Test_1.predict( np.reshape( temp_test_data[i],(1,3,1,1)) ) )

2 2 [[0.33467758 0.32697734 0.3383451 ]]
2 2 [[0.3352379  0.32642987 0.33833227]]
2 2 [[0.33552605 0.32633337 0.33814055]]
2 0 [[0.33659557 0.3254965  0.33790794]]
0 0 [[0.33744225 0.32513276 0.33742493]]
0 0 [[0.33807394 0.32500312 0.3369229 ]]
0 0 [[0.33871442 0.32477906 0.33650655]]
0 1 [[0.33877978 0.32458612 0.33663407]]
2 1 [[0.3358184  0.32644936 0.33773226]]
2 0 [[0.336579   0.3258266  0.33759445]]
2 0 [[0.33465168 0.32701746 0.33833084]]
2 0 [[0.33416185 0.32719752 0.3386406 ]]
2 2 [[0.33344683 0.32743597 0.3391172 ]]
2 1 [[0.33365658 0.32741925 0.33892414]]
2 1 [[0.33385333 0.32712492 0.33902177]]
2 1 [[0.33460706 0.32660583 0.3387871 ]]
2 0 [[0.33479774 0.32631338 0.3388889 ]]
2 0 [[0.33379236 0.32658672 0.33962095]]
2 0 [[0.33294848 0.32677835 0.34027317]]
2 1 [[0.3335055  0.32738745 0.33910704]]
2 1 [[0.33358204 0.327148   0.33927   ]]
2 1 [[0.3340851  0.32672545 0.33918944]]
2 1 [[0.33402255 0.326565   0.33941242]]
2 1 [[0.3343384  0.3262549  0.33940667]]
2 1 [[0.33299989

In [ ]:
 # temp_loaded.summary()

In [ ]:
temp_proba=temp_test_data
temp_label=test_label_raw


reste=False
while reste==False :
    nbr=0
    total_rencotrer=0
    taille_vecteur=[]
    tableau_eliminer=[]
    for i in range(1,len(temp_proba)-25) :

        total=0

        indice =  np.argmax( model_Test_1.predict( np.reshape( temp_proba[i],(1,37,1,1)) ) )+1
        # indice = 1

        if np.argmin(temp_label[i-1]) != 10000 :

            condition=True
            cpt=0
            while condition == True and cpt<15 :
                if indice-1 != np.argmin(temp_label[i+cpt ]) :
                    total+= 1
                    cpt+=1
                else :
                    condition=False

            if total > 7 :

                nbr+=1


            total_rencotrer+=1
    if nbr%1 == 0 :
        reste=True
    print(nbr, total_rencotrer, round(100*nbr/total_rencotrer,4))


In [ ]:
#14 974 1.4374


In [ ]:
model_temp=model_Test_1

model_temp.save("model_predict.h5" )

In [ ]:
temp_loaded= tf.keras.models.load_model("model_predict.h5")

In [ ]:
test_loss, test_accuracy = temp_loaded.evaluate( test_dataset , steps=math.ceil(len(test_data)/BACTH_SIZE))
print('Accuracy on test dataset:', test_accuracy )

In [ ]:
x=0.35
n=1
n0=100
formule=0
for i in range(1,n+1) :
    # print(i)
    formule += n0*math.pow(2,i)*math.pow(x,i) - n0*math.pow(2,i-1)*math.pow(1-x,i)
print(formule)

In [ ]:
a=[1,2,3]
# a/=9